# Challenge 3 (Bonus)

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from [here](https://drive.google.com/file/d/17oJ9NNbwJ8Wv4Jp-mLFiC59k72Brdyha/view?usp=sharing) and place it in the provided data folder. You will access the data folder with a relative path.
Load `Orders` into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here

orders = pd.read_csv("data/Orders.csv", sep=",")
#orders.columns
#Renaming first column.. was disturbing :D - Or I believe I can simply drop that one since first column is also index #s.
orders.columns = ['Index No', 'InvoiceNo', 'StockCode', 'year', 'month', 'day', 'hour','Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID','Country', 'amount_spent']

orders.head()

,Index No,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here

# Starting labeling customers based on their expenses
# Groupby CustomerID to see total expenses for each customer
# 75%-95% --> Preferred
#  > 95% ---> VIP

customerID_spent = orders[["amount_spent","CustomerID"]].groupby("CustomerID").sum().sort_values(by="amount_spent",ascending=False)

#customerID_spent["amount_spent"].mean()
#customerID_spent.describe()

#pd.qcut(customerID_spent["total_amount_spent"], q = [0,.25,.50,.75,.95,1],labels=["Low","Med","High","Preferred","VIP"]).value_counts().sort_values(ascending=False)

customerID_spent["Customer Profiles"] = pd.qcut(customerID_spent["amount_spent"], q = [0,.25,.50,.75,.95,1],labels=["Low","Med","High","Preferred","VIP"])#.value_counts().sort_values(ascending=False)


# To see results and edges..

results, bin_edges = pd.qcut(customerID_spent["amount_spent"], q = [0,.25,.50,.75,.95,1],labels=["Low","Med","High","Preferred","VIP"],retbins=True)
#print(results)
print(bin_edges)
customerID_spent 

[     0.       307.245    674.45    1661.64    5840.182 280206.02 ]


,amount_spent,Customer Profiles
CustomerID,,
14646,280206.02,VIP
18102,259657.30,VIP
17450,194550.79,VIP
16446,168472.50,VIP
14911,143825.06,VIP
...,...,...
17956,12.75,Low
16454,6.90,Low
14792,6.20,Low


In [4]:
#To have clear idea and well structured table - also answer to Sub Problem 3

label_names=["Low","Med","High","Preferred","VIP"]
label_table = pd.DataFrame(zip(bin_edges,label_names),columns=["Limits","Customer Types"])
label_table

,Limits,Customer Types
0,0.000,Low
1,307.245,Med
2,674.450,High
3,1661.640,Preferred
4,5840.182,VIP


In [5]:
# Sub problem 2 - how to choose customers in a given quantile range

# Now, since we already labeled which ranges corresponds to which type of customer;

VIP_Customers = customerID_spent[customerID_spent["Customer Profiles"] == "VIP"]
VIP_Customers

,amount_spent,Customer Profiles
CustomerID,,
14646,280206.02,VIP
18102,259657.30,VIP
17450,194550.79,VIP
16446,168472.50,VIP
14911,143825.06,VIP
...,...,...
14505,5905.64,VIP
14415,5900.28,VIP
14944,5900.25,VIP


In [6]:
Preferred_Customers = customerID_spent[customerID_spent["Customer Profiles"] == "Preferred"]
Preferred_Customers

,amount_spent,Customer Profiles
CustomerID,,
13050,5836.86,Preferred
12720,5781.73,Preferred
15218,5756.89,Preferred
17686,5739.46,Preferred
13178,5725.47,Preferred
...,...,...
17656,1674.69,Preferred
16115,1667.97,Preferred
12912,1662.30,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [7]:
# So we have 2 distinct tables at this moment, 
# a) Our initial orders table
orders

,Index No,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


In [8]:
# b) Our labeled table based on customer ID

# So we have to merge them based on customerID_spent first;

# Here I renamed "amount_spent" in customerID_spent table to "total_amount_spent" to avoid confusion while merging

customerID_spent.columns = ['total_amount_spent', 'Customer Profiles']
customerID_spent

,total_amount_spent,Customer Profiles
CustomerID,,
14646,280206.02,VIP
18102,259657.30,VIP
17450,194550.79,VIP
16446,168472.50,VIP
14911,143825.06,VIP
...,...,...
17956,12.75,Low
16454,6.90,Low
14792,6.20,Low


In [9]:
# After execution of code, we have one table, all data we need inside it

final_df = pd.merge(orders, customerID_spent, on=['CustomerID'])
final_df

,Index No,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,total_amount_spent,Customer Profiles
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,5391.21,Preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,5391.21,Preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541801,581578,22993,2011,12,5,12,set of 4 pantry jelly moulds,12,2011-12-09 12:16:00,1.25,12713,Germany,15.00,848.55,High
397920,541802,581578,22907,2011,12,5,12,pack of 20 napkins pantry design,12,2011-12-09 12:16:00,0.85,12713,Germany,10.20,848.55,High
397921,541803,581578,22908,2011,12,5,12,pack of 20 napkins red apples,12,2011-12-09 12:16:00,0.85,12713,Germany,10.20,848.55,High
397922,541804,581578,23215,2011,12,5,12,jingle bell heart antique silver,12,2011-12-09 12:16:00,2.08,12713,Germany,24.96,848.55,High


In [10]:
final_df.columns

Index(['Index No', 'InvoiceNo', 'StockCode', 'year', 'month', 'day', 'hour',
       'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID',
       'Country', 'amount_spent', 'total_amount_spent', 'Customer Profiles'],
      dtype='object')

In [11]:
# If we group by Country and count values, we can find which country has most VIP customer

final_df.drop(columns=['Index No',"total_amount_spent","amount_spent","CustomerID", 'InvoiceNo', 'StockCode', 'year', 'month', 'day', 'hour','Description', 'Quantity', 'InvoiceDate', 'UnitPrice'],inplace=True)

final_df[final_df["Customer Profiles"].isin(["VIP"])].groupby("Country").count().sort_values(by="Customer Profiles",ascending=False)

# Most VIP customers belong to United Kingdom in this dataframe

,Customer Profiles
Country,
United Kingdom,84185
EIRE,7077
France,3290
Germany,3127
Netherlands,2080
Australia,898
Portugal,681
Switzerland,594
Spain,511


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [12]:
# your code here

# By simply adding "Preferred" customer type into our isin() function we can see VIP+Preferred Customers

final_df[final_df["Customer Profiles"].isin(["VIP","Preferred"])].groupby("Country").count().sort_values(by="Customer Profiles",ascending=False)

# In this case, again UK has most VIP+Preferred customers

,Customer Profiles
Country,
United Kingdom,221635
Germany,7349
EIRE,7238
France,6301
Netherlands,2080
Spain,1569
Belgium,1557
Switzerland,1370
Portugal,1093


### Sorry for messy codes, need some time to get familiar with Pandas, and then I will able to write more cleaner codes :)